In [1]:
from source.load_excel_to_table import load_excel_to_table

part_a_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "甲方", "part_a")
part_b_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "乙方", "part_b")

In [2]:
from source.get_group_conditions import get_group_conditions


part_a_group_col_rst = get_group_conditions(part_a_conn, "part_a", "统一仓库", "统一名称", "统一日期")
part_b_group_col_rst = get_group_conditions(part_b_conn, "part_b", "统一仓库", "统一名称", "统一日期")

In [ ]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data
from collections import Counter

list_a = get_group_data(part_a_conn, "part_a", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-12-01 00:00:00", "收货数量")
list_b = get_group_data(part_b_conn, "part_b", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-12-01 00:00:00", "实发数量")
_rst = reconcile_lists(list_a, list_b)
rich.print(_rst)
print(Counter(_rst["unmatched_a"]))
print(Counter(_rst["unmatched_b"])) 

{
    'exact_matches': {1.0: 1, 20.0: 2, 23.0: 1, 25.0: 1, 28.0: 1},
    'combo_matches_a_to_b': [
        {'b': 100.0, 'a': [28.0, 28.0, 28.0, 16.0]},
        {'b': 80.0, 'a': [28.0, 28.0, 24.0]},
        {'b': 76.0, 'a': [28.0, 28.0, 16.0, 4.0]},
        {'b': 52.0, 'a': [28.0, 24.0]},
        {'b': 50.0, 'a': [26.0, 16.0, 8.0]},
        {'b': 35.0, 'a': [28.0, 5.0, 2.0]},
        {'b': 30.0, 'a': [15.0, 15.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 16.0, 12.0, 12.0, 12.0, 12.0, 8.0, 4.0, 1.0],
    'unmatched_b': [100.0, 100.0, 100.0]
}

Counter({28.0: 8, 12.0: 4, 16.0: 1, 8.0: 1, 4.0: 1, 1.0: 1})
Counter({100.0: 3})


In [ ]:
for group1 in part_a_group_col_rst["group1_col"]:
    for group2 in part_a_group_col_rst["group2_col"]:
        for group3 in part_a_group_col_rst["group3_col"]:
            list_a = get_group_data(part_a_conn, "part_a", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "收货数量")
            list_b = get_group_data(part_b_conn, "part_b", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "实发数量")
            rst = reconcile_lists(list_a, list_b)
            print(rst)